In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
import shutil

# Define your Google Drive path
base_path = '/content/drive/MyDrive/Colab Notebooks/Deep Learning/image_recognition_project'

# Create folder structure
folders = [
    'dataset/sample_100',
    'dataset/full_collection',
    'reference_faces',
    'output/matched',
    'output/not_matched',
    'output/review',
    'logs'
]

print("Creating folder structure...")
print("="*70)

for folder in folders:
    folder_path = os.path.join(base_path, folder)
    os.makedirs(folder_path, exist_ok=True)
    print(f"✓ Created: {folder}")

print("="*70)
print("✅ Folder structure created successfully!")
print("\n📂 Your project structure:")
print(f"\n{base_path}/")
print("├── dataset/")
print("│   ├── sample_100/           ← Upload 100 test photos here")
print("│   └── full_collection/      ← Upload all 5K photos here (later)")
print("├── reference_faces/          ← Upload 5-10 photos of YOURSELF here")
print("├── output/")
print("│   ├── matched/              ← Photos with you (auto-generated)")
print("│   ├── not_matched/          ← Photos without you (auto-generated)")
print("│   └── review/               ← Borderline cases (auto-generated)")
print("└── logs/                     ← Processing logs (auto-generated)")

print("\n" + "="*70)
print("📌 NEXT STEPS:")
print("="*70)
print("1. Go to your Google Drive")
print(f"2. Navigate to: Colab Notebooks/Deep Learning/image_recognition_project")
print("3. Upload files to these folders:")
print("   • reference_faces/ → 5-10 clear photos of YOUR face")
print("   • dataset/sample_100/ → 100 diverse test photos")
print("\n4. After uploading, come back to Colab and continue with the code")
print("="*70)

In [ ]:
def check_folder_structure(base_path):
    """
    Check and display the folder structure and file counts
    """
    print("\n" + "="*70)
    print("📁 FOLDER STRUCTURE VERIFICATION")
    print("="*70)

    # Check reference faces
    ref_folder = os.path.join(base_path, 'reference_faces')
    ref_files = [f for f in os.listdir(ref_folder)
                 if f.lower().endswith(('.jpg', '.jpeg', '.png'))]

    print(f"\n📸 Reference Faces: {ref_folder}")
    print(f"   Files found: {len(ref_files)}")
    if len(ref_files) > 0:
        print("   Sample files:")
        for f in ref_files[:5]:
            print(f"      • {f}")
    else:
        print("   ⚠️ WARNING: No reference photos found!")

    # Check sample dataset
    sample_folder = os.path.join(base_path, 'dataset/sample_100')
    sample_files = [f for f in os.listdir(sample_folder)
                    if f.lower().endswith(('.jpg', '.jpeg', '.png'))]

    print(f"\n📷 Sample Dataset: {sample_folder}")
    print(f"   Files found: {len(sample_files)}")
    if len(sample_files) > 0:
        print("   Sample files:")
        for f in sample_files[:5]:
            print(f"      • {f}")
        if len(sample_files) > 5:
            print(f"      ... and {len(sample_files) - 5} more")
    else:
        print("   ⚠️ WARNING: No test photos found!")

    # Check output folders
    output_folders = ['matched', 'not_matched', 'review']
    print(f"\n📂 Output Folders:")
    for folder in output_folders:
        folder_path = os.path.join(base_path, 'output', folder)
        if os.path.exists(folder_path):
            print(f"   ✓ {folder}/")
        else:
            print(f"   ✗ {folder}/ (missing)")

    print("\n" + "="*70)

    # Summary
    if len(ref_files) > 0 and len(sample_files) > 0:
        print("✅ STATUS: Ready to proceed with processing!")
        print(f"   • Reference faces: {len(ref_files)} photos")
        print(f"   • Test dataset: {len(sample_files)} photos")
    else:
        print("⚠️ STATUS: Please upload files before proceeding")
        if len(ref_files) == 0:
            print("   • Missing: Reference photos of yourself")
        if len(sample_files) == 0:
            print("   • Missing: Test photos in sample_100")

    print("="*70)

    return len(ref_files), len(sample_files)

# Run verification
base_path = '/content/drive/MyDrive/Colab Notebooks/Deep Learning/image_recognition_project'
num_ref, num_sample = check_folder_structure(base_path)

In [ ]:
# Install face_recognition library
print("Installing required libraries...")
print("="*70)

!pip install -q face_recognition

print("\n✅ Installation complete!")
print("="*70)

In [ ]:
# Import all necessary libraries
import face_recognition
import cv2
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from datetime import datetime
import pandas as pd
from tqdm import tqdm
import json
import os
import shutil

print("✅ All libraries imported successfully!")

In [ ]:
def load_reference_faces(reference_folder):
    """
    Load reference images and create face encodings

    Returns:
        reference_encodings: List of face encodings from reference images
        reference_names: List of filenames for tracking
    """
    reference_encodings = []
    reference_names = []

    # Get all image files from reference folder
    image_extensions = ['.jpg', '.jpeg', '.png', '.JPG', '.JPEG', '.PNG']
    reference_files = [f for f in os.listdir(reference_folder)
                      if any(f.endswith(ext) for ext in image_extensions)]

    if len(reference_files) == 0:
        print("⚠️ No reference images found!")
        return [], []

    print("="*70)
    print(f"LOADING REFERENCE FACES")
    print("="*70)
    print(f"Processing {len(reference_files)} reference images...\n")

    for filename in reference_files:
        file_path = os.path.join(reference_folder, filename)

        try:
            # Load image
            image = face_recognition.load_image_file(file_path)

            # Get face encodings
            encodings = face_recognition.face_encodings(image)

            if len(encodings) > 0:
                # Use the first face found
                reference_encodings.append(encodings[0])
                reference_names.append(filename)
                print(f"  ✓ {filename}: Face encoded successfully")
            else:
                print(f"  ✗ {filename}: No face detected")

        except Exception as e:
            print(f"  ✗ {filename}: Error - {str(e)}")

    print("\n" + "="*70)
    print(f"✅ Successfully encoded {len(reference_encodings)} reference faces")
    print(f"   Each encoding has {len(reference_encodings[0])} dimensions")
    print("="*70 + "\n")

    return reference_encodings, reference_names


# Load your reference faces
base_path = '/content/drive/MyDrive/Colab Notebooks/Deep Learning/image_recognition_project'
reference_folder = os.path.join(base_path, 'reference_faces')
reference_encodings, reference_names = load_reference_faces(reference_folder)

In [ ]:
def visualize_reference_faces(reference_folder, reference_names):
    """
    Display all reference faces to verify they loaded correctly
    """
    num_images = len(reference_names)

    if num_images == 0:
        print("No reference images to display")
        return

    # Calculate grid size
    cols = min(5, num_images)
    rows = (num_images + cols - 1) // cols

    fig, axes = plt.subplots(rows, cols, figsize=(15, 3*rows))

    # Flatten axes array for easier indexing
    if num_images == 1:
        axes = [axes]
    else:
        axes = axes.flatten() if num_images > cols else axes

    for idx, filename in enumerate(reference_names):
        file_path = os.path.join(reference_folder, filename)
        img = Image.open(file_path)
        axes[idx].imshow(img)
        axes[idx].set_title(filename, fontsize=10)
        axes[idx].axis('off')

    # Hide extra subplots if any
    for idx in range(num_images, len(axes) if isinstance(axes, np.ndarray) else 1):
        if isinstance(axes, np.ndarray):
            axes[idx].axis('off')

    plt.suptitle("✅ Your Reference Faces (These will be used for matching)",
                 fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.show()

# Visualize your reference faces
visualize_reference_faces(reference_folder, reference_names)

In [ ]:
def process_single_image(image_path, reference_encodings, tolerance=0.6):
    """
    Process a single image and determine if it contains the reference person

    Args:
        image_path: Path to the image file
        reference_encodings: List of reference face encodings
        tolerance: Distance threshold (lower = more strict)

    Returns:
        dict with results
    """
    result = {
        'filename': os.path.basename(image_path),
        'faces_detected': 0,
        'min_distance': None,
        'is_match': False,
        'classification': 'error',
        'error': None
    }

    try:
        # Load image
        image = face_recognition.load_image_file(image_path)

        # Detect faces and get encodings
        face_locations = face_recognition.face_locations(image)
        face_encodings = face_recognition.face_encodings(image, face_locations)

        result['faces_detected'] = len(face_encodings)

        if len(face_encodings) == 0:
            result['classification'] = 'not_matched'  # No faces = not matched
            return result

        # Compare each detected face with reference faces
        all_distances = []

        for face_encoding in face_encodings:
            # Calculate distance to each reference encoding
            distances = face_recognition.face_distance(reference_encodings, face_encoding)
            min_dist = np.min(distances)
            all_distances.append(min_dist)

        # Get the minimum distance across all faces in the image
        result['min_distance'] = float(np.min(all_distances))

        # Classify based on threshold
        if result['min_distance'] < tolerance:
            result['is_match'] = True
            result['classification'] = 'matched'
        elif result['min_distance'] < tolerance + 0.1:  # Borderline cases
            result['classification'] = 'review'
        else:
            result['classification'] = 'not_matched'

    except Exception as e:
        result['error'] = str(e)
        result['classification'] = 'error'

    return result


def process_dataset(input_folder, output_base_folder, reference_encodings,
                   tolerance=0.6, copy_files=True):
    """
    Process all images in a folder

    Args:
        input_folder: Folder containing images to process
        output_base_folder: Base folder for outputs
        reference_encodings: Reference face encodings
        tolerance: Matching threshold
        copy_files: Whether to copy files to output folders

    Returns:
        DataFrame with all results
    """
    # Get all image files
    image_extensions = ['.jpg', '.jpeg', '.png', '.JPG', '.JPEG', '.PNG', '.heic', '.HEIC']
    image_files = [f for f in os.listdir(input_folder)
                   if any(f.endswith(ext) for ext in image_extensions)]

    print("\n" + "="*70)
    print("STARTING IMAGE PROCESSING")
    print("="*70)
    print(f"📁 Input folder: {input_folder}")
    print(f"📊 Found {len(image_files)} images to process")
    print(f"🎯 Tolerance: {tolerance}")
    print(f"👤 Reference faces: {len(reference_encodings)}")
    print("="*70 + "\n")

    if len(image_files) == 0:
        print("⚠️ No images found in input folder!")
        return pd.DataFrame()

    # Process each image
    results = []

    for filename in tqdm(image_files, desc="Processing images", unit="image"):
        image_path = os.path.join(input_folder, filename)

        # Process the image
        result = process_single_image(image_path, reference_encodings, tolerance)
        result['timestamp'] = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        results.append(result)

        # Copy file to appropriate folder
        if copy_files:
            classification = result['classification']
            dest_folder = os.path.join(output_base_folder, classification)
            dest_path = os.path.join(dest_folder, filename)

            try:
                shutil.copy2(image_path, dest_path)
            except Exception as e:
                print(f"\n⚠️ Error copying {filename}: {e}")

    # Create DataFrame
    results_df = pd.DataFrame(results)

    return results_df

In [ ]:
# Define paths
base_path = '/content/drive/MyDrive/Colab Notebooks/Deep Learning/image_recognition_project'
input_folder = os.path.join(base_path, 'dataset/sample_100')
output_folder = os.path.join(base_path, 'output')

# Set tolerance
# Lower = more strict (fewer false positives, might miss some photos of you)
# Higher = more lenient (catches more photos of you, might have false positives)
# Recommended range: 0.4 to 0.6
tolerance = 0.6

print("⚙️ CONFIGURATION:")
print("="*70)
print(f"Tolerance: {tolerance}")
print(f"Reference faces loaded: {len(reference_encodings)}")
print(f"Input folder: {input_folder}")
print(f"Output folder: {output_folder}")
print("="*70)

# Process all images
results_df = process_dataset(
    input_folder=input_folder,
    output_base_folder=output_folder,
    reference_encodings=reference_encodings,
    tolerance=tolerance,
    copy_files=True
)

In [ ]:
def analyze_results(results_df, tolerance):
    """Display comprehensive analysis of results"""

    print("\n" + "="*70)
    print("📊 PROCESSING RESULTS - SUMMARY")
    print("="*70)

    print(f"\n🔢 Total Statistics:")
    print(f"   • Total images processed: {len(results_df)}")
    print(f"   • Tolerance used: {tolerance}")

    print(f"\n📋 Classification Breakdown:")
    classification_counts = results_df['classification'].value_counts()
    for classification, count in classification_counts.items():
        percentage = (count / len(results_df)) * 100
        emoji = "✅" if classification == "matched" else "❌" if classification == "not_matched" else "⚠️" if classification == "review" else "🔴"
        print(f"   {emoji} {classification.upper()}: {count} ({percentage:.1f}%)")

    print(f"\n👤 Face Detection:")
    print(f"   • Images with faces detected: {len(results_df[results_df['faces_detected'] > 0])}")
    print(f"   • Images without faces: {len(results_df[results_df['faces_detected'] == 0])}")
    print(f"   • Total faces detected: {results_df['faces_detected'].sum()}")

    # Distance statistics (only for images with faces)
    matched_df = results_df[results_df['min_distance'].notna()]
    if len(matched_df) > 0:
        print(f"\n📏 Distance Statistics (lower = more similar):")
        print(f"   • Mean distance: {matched_df['min_distance'].mean():.4f}")
        print(f"   • Min distance: {matched_df['min_distance'].min():.4f}")
        print(f"   • Max distance: {matched_df['min_distance'].max():.4f}")
        print(f"   • Median distance: {matched_df['min_distance'].median():.4f}")

        # Show distance ranges
        matched_count = len(matched_df[matched_df['classification'] == 'matched'])
        review_count = len(matched_df[matched_df['classification'] == 'review'])
        not_matched_count = len(matched_df[matched_df['classification'] == 'not_matched'])

        print(f"\n📊 Distance Ranges:")
        if matched_count > 0:
            matched_distances = matched_df[matched_df['classification'] == 'matched']['min_distance']
            print(f"   • MATCHED (< {tolerance}): {matched_distances.min():.4f} to {matched_distances.max():.4f}")
        if review_count > 0:
            review_distances = matched_df[matched_df['classification'] == 'review']['min_distance']
            print(f"   • REVIEW ({tolerance} to {tolerance+0.1}): {review_distances.min():.4f} to {review_distances.max():.4f}")
        if not_matched_count > 0:
            not_matched_distances = matched_df[matched_df['classification'] == 'not_matched']['min_distance']
            print(f"   • NOT MATCHED (≥ {tolerance+0.1}): {not_matched_distances.min():.4f} to {not_matched_distances.max():.4f}")

    # Errors
    errors = results_df[results_df['classification'] == 'error']
    if len(errors) > 0:
        print(f"\n⚠️ Errors Encountered: {len(errors)}")
        print("\nError Details:")
        for idx, row in errors.iterrows():
            print(f"   • {row['filename']}: {row['error']}")

    print("\n" + "="*70)
    print("✅ ANALYSIS COMPLETE")
    print("="*70)

# Run analysis
analyze_results(results_df, tolerance)

In [ ]:
def visualize_distance_distribution(results_df, tolerance):
    """Plot distance distribution and classification breakdown"""

    matched_df = results_df[results_df['min_distance'].notna()]

    if len(matched_df) == 0:
        print("⚠️ No faces detected in any images!")
        return

    fig = plt.figure(figsize=(16, 5))

    # 1. Histogram of distances
    plt.subplot(1, 3, 1)
    plt.hist(matched_df['min_distance'], bins=30, edgecolor='black', alpha=0.7, color='skyblue')
    plt.axvline(tolerance, color='red', linestyle='--', linewidth=2,
                label=f'Threshold: {tolerance}')
    plt.axvline(tolerance + 0.1, color='orange', linestyle='--', linewidth=2,
                label=f'Review boundary: {tolerance + 0.1}')
    plt.xlabel('Distance', fontsize=12)
    plt.ylabel('Frequency', fontsize=12)
    plt.title('Distribution of Face Distances', fontsize=14, fontweight='bold')
    plt.legend()
    plt.grid(True, alpha=0.3)

    # 2. Box plot by classification
    plt.subplot(1, 3, 2)
    classifications = ['matched', 'review', 'not_matched']
    data_to_plot = []
    labels_to_plot = []

    for c in classifications:
        if c in matched_df['classification'].values:
            data_to_plot.append(matched_df[matched_df['classification'] == c]['min_distance'].values)
            labels_to_plot.append(c)

    if data_to_plot:
        bp = plt.boxplot(data_to_plot, labels=labels_to_plot, patch_artist=True)
        colors = ['lightgreen', 'yellow', 'lightcoral']
        for patch, color in zip(bp['boxes'], colors[:len(bp['boxes'])]):
            patch.set_facecolor(color)

        plt.axhline(tolerance, color='red', linestyle='--', linewidth=2,
                   label=f'Threshold: {tolerance}')
        plt.ylabel('Distance', fontsize=12)
        plt.title('Distance by Classification', fontsize=14, fontweight='bold')
        plt.legend()
        plt.grid(True, alpha=0.3, axis='y')

    # 3. Pie chart of classifications
    plt.subplot(1, 3, 3)
    classification_counts = results_df['classification'].value_counts()
    colors_pie = {'matched': 'lightgreen', 'review': 'yellow',
                  'not_matched': 'lightcoral', 'error': 'gray'}
    colors_list = [colors_pie.get(label, 'lightgray') for label in classification_counts.index]

    plt.pie(classification_counts.values, labels=classification_counts.index,
            autopct='%1.1f%%', startangle=90, colors=colors_list)
    plt.title('Classification Distribution', fontsize=14, fontweight='bold')

    plt.tight_layout()
    plt.show()

# Visualize results
visualize_distance_distribution(results_df, tolerance)

In [ ]:
def show_sample_results(results_df, output_folder, classification='matched', num_samples=6):
    """
    Display sample images from a specific classification
    """
    samples = results_df[results_df['classification'] == classification].head(num_samples)

    if len(samples) == 0:
        print(f"ℹ️ No images in '{classification}' category")
        return

    num_images = len(samples)
    cols = min(3, num_images)
    rows = (num_images + cols - 1) // cols

    fig, axes = plt.subplots(rows, cols, figsize=(15, 5*rows))

    if num_images == 1:
        axes = [axes]
    else:
        axes = axes.flatten()

    for idx, (_, row) in enumerate(samples.iterrows()):
        img_path = os.path.join(output_folder, classification, row['filename'])

        if os.path.exists(img_path):
            img = Image.open(img_path)
            axes[idx].imshow(img)

            # Create title with info
            if row['min_distance']:
                title = f"{row['filename']}\nDistance: {row['min_distance']:.3f} | Faces: {row['faces_detected']}"
            else:
                title = f"{row['filename']}\nNo faces detected"

            axes[idx].set_title(title, fontsize=9)
            axes[idx].axis('off')

    # Hide extra subplots
    for idx in range(num_images, len(axes)):
        axes[idx].axis('off')

    emoji = "✅" if classification == "matched" else "❌" if classification == "not_matched" else "⚠️"
    plt.suptitle(f"{emoji} Sample {classification.upper()} Images",
                 fontsize=16, fontweight='bold')
    plt.tight_layout()
    plt.show()

# Show matched images
print("📸 SAMPLE MATCHED IMAGES (Photos containing YOU):")
print("="*70)
show_sample_results(results_df, output_folder, 'matched', num_samples=6)

print("\n📸 SAMPLE NOT MATCHED IMAGES (Photos without you):")
print("="*70)
show_sample_results(results_df, output_folder, 'not_matched', num_samples=6)

if len(results_df[results_df['classification'] == 'review']) > 0:
    print("\n📸 SAMPLE REVIEW IMAGES (Borderline cases - please check manually):")
    print("="*70)
    show_sample_results(results_df, output_folder, 'review', num_samples=6)

In [ ]:
def save_results(results_df, base_path, tolerance, num_references):
    """Save results to CSV and JSON"""

    print("\n" + "="*70)
    print("💾 SAVING RESULTS")
    print("="*70)

    # Save detailed results CSV
    results_csv_path = os.path.join(base_path, 'logs', 'results.csv')
    results_df.to_csv(results_csv_path, index=False)
    print(f"✓ Detailed results saved to: results.csv")

    # Save summary JSON
    summary = {
        'total_images': len(results_df),
        'classification_counts': results_df['classification'].value_counts().to_dict(),
        'faces_detected_total': int(results_df['faces_detected'].sum()),
        'images_with_faces': len(results_df[results_df['faces_detected'] > 0]),
        'tolerance_used': tolerance,
        'reference_faces_count': num_references,
        'processing_date': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
        'distance_statistics': {
            'mean': float(results_df[results_df['min_distance'].notna()]['min_distance'].mean()) if len(results_df[results_df['min_distance'].notna()]) > 0 else None,
            'min': float(results_df[results_df['min_distance'].notna()]['min_distance'].min()) if len(results_df[results_df['min_distance'].notna()]) > 0 else None,
            'max': float(results_df[results_df['min_distance'].notna()]['min_distance'].max()) if len(results_df[results_df['min_distance'].notna()]) > 0 else None,
        }
    }

    summary_path = os.path.join(base_path, 'logs', 'summary.json')
    with open(summary_path, 'w') as f:
        json.dump(summary, f, indent=2)
    print(f"✓ Summary saved to: summary.json")

    print("\n📂 All results saved in:")
    print(f"   {os.path.join(base_path, 'logs')}/")
    print("="*70)

# Save results
save_results(results_df, base_path, tolerance, len(reference_encodings))

In [ ]:
print("\n" + "="*70)
print("📋 SAMPLE RESULTS TABLE")
print("="*70)
print("\nFirst 10 results:")
display(results_df.head(10))

print("\nLast 10 results:")
display(results_df.tail(10))

In [ ]:
print("\n" + "="*70)
print("🎉 PROCESSING COMPLETE!")
print("="*70)

print(f"\n📊 Final Summary:")
print(f"   • Total images processed: {len(results_df)}")
print(f"   • Matched (contain you): {len(results_df[results_df['classification'] == 'matched'])}")
print(f"   • Not matched: {len(results_df[results_df['classification'] == 'not_matched'])}")
print(f"   • Review needed: {len(results_df[results_df['classification'] == 'review'])}")
print(f"   • Errors: {len(results_df[results_df['classification'] == 'error'])}")

print(f"\n📁 Check your results in Google Drive:")
print(f"   Colab Notebooks/Deep Learning/image_recognition_project/output/")
print(f"   • matched/ - Photos containing you")
print(f"   • not_matched/ - Photos without you")
print(f"   • review/ - Borderline cases (check manually)")

print(f"\n⚙️ Next Steps:")
print("="*70)
print("1. ✅ Review the 'matched' folder - Are these correct?")
print("2. ✅ Review the 'review' folder - Manually classify these")
print("3. ✅ Check 'not_matched' - Any photos of you missed?")
print("\n4. If results are GOOD:")
print("   → You're ready to process your full 5K collection!")
print("\n5. If results need improvement:")
print("   → Adjust tolerance value (try 0.5 for stricter or 0.7 for lenient)")
print("   → Add more reference photos")
print("   → Run processing again")
print("="*70)